# Assignment 3

**Due Thursday, Oct 8 2020 @ 11:55 pm**

Calculate the forces in each member. Assume AE is constant. Green arrows show far/end nodes. **Show all work.**

<img src="https://storage.googleapis.com/nm-static/computing_maloof2_20201001.png" alt="nyu_comp_f20">


In [15]:
# Imports
import numpy as np

In [16]:
# Establish coordinates
x1, y1 = 4, 4
x2, y2 = 0, 0
x3, y3 = 4, 0
x4, y4 = 7, 0

In [17]:
# Calculate member lengths
def member_len(xy1, xy2):
    """ Given the coordinates of two points, the distance between them
    will be calculated.
    """
    return np.linalg.norm(np.array(xy1) - np.array(xy2))

mem1 = member_len((x1, y1), (x2, y2))
mem2 = member_len((x1, y1), (x3, y3))
mem3 = member_len((x1, y1), (x4, y4))

print('Length of Member 1:', mem1, '\nLength of Member 2:', mem2, '\nLength of Member 3:', mem3)

Length of Member 1: 5.656854249492381 
Length of Member 2: 4.0 
Length of Member 3: 5.0


In [18]:
# Finding direction cosines

# Member 1 (nodes 1 and 2)
mem_x1 = (x2 - x1)/mem1
mem_y1 = (y2 - y1)/mem1

# Member 2 (nodes 1 and 3)
mem_x2 = (x3 - x1)/mem2
mem_y2 = (y3 - y1)/mem2

# Member 3 (nodes 1 and 4)
mem_x3 = (x4 - x1)/mem3
mem_y3 = (y4 - y1)/mem3

print('Member 1:', mem_x1, mem_y1, '\nMember 2:', mem_x2, mem_y2, '\nMember 3:', mem_x3, mem_y3)

Member 1: -0.7071067811865475 -0.7071067811865475 
Member 2: 0.0 -1.0 
Member 3: 0.6 -0.8


In [19]:
# Finding direction cosines (function)
def dir_cos(xy1, xy2):
    """ Given the point coordinates for a member, its direction cosines will be calculated.
    """
    mem_len = member_len(xy1, xy2)
    return (xy2[0] - xy1[0])/mem_len, (xy2[1] - xy1[1])/mem_len

mem_x1, mem_y1 = dir_cos((x1, y1), (x2, y2))
mem_x2, mem_y2 = dir_cos((x1, y1), (x3, y3))
mem_x3, mem_y3 = dir_cos((x1, y1), (x4, y4))

print('Member 1:', mem_x1, mem_y1, '\nMember 2:', mem_x2, mem_y2, '\nMember 3:', mem_x3, mem_y3)

Member 1: -0.7071067811865475 -0.7071067811865475 
Member 2: 0.0 -1.0 
Member 3: 0.6 -0.8


The Stiffness Matrix

$$
K_j = \dfrac{AE}{L}
\begin{bmatrix}
(\lambda x)^2 & \lambda x \lambda y & -(\lambda x)^2 & - \lambda x \lambda y \\
\lambda x \lambda y & (\lambda y)^2 & -\lambda x \lambda y & -(\lambda y)^2 \\
-(\lambda x)^2 & -\lambda x \lambda y & (\lambda x)^2 & \lambda x \lambda y \\
-\lambda x \lambda y & -(\lambda y)^2 & \lambda x \lambda y & (\lambda y)^2
\end{bmatrix}
$$

It is assumed that AE is constant.

In [20]:
# In order to determine the stiffness matrix for a member, the matrix is divided by the length of the member.
def stiff_mtx(xy1, xy2):
    """ Given the coordinates of a member's nodes, the stiffness matrix for that member will be returned.
    """
    memlen = member_len(xy1, xy2)
    lmx, lmy = dir_cos(xy1, xy2)
    """ The past two functions are recalled and their results are stored in variables for easy access.
    """
    return np.array(
        [
        [lmx**2, lmx*lmy, -lmx**2, -lmx*lmy],
        [lmx*lmy, lmy**2, -lmx*lmy, -lmy**2],
        [-lmx**2, -lmx*lmy, lmx**2, lmx*lmy],
        [-lmx*lmy, -lmy**2, lmx*lmy, lmy**2]
        ]
    ) / memlen

k1 = stiff_mtx((x1, y1), (x2, y2))
k2 = stiff_mtx((x1, y1), (x3, y3))
k3 = stiff_mtx((x1, y1), (x4, y4))

print('Member 1:', '\n', k1, '\n')
print('Member 2:', '\n', k2, '\n')
print('Member 3:', '\n', k3)

Member 1: 
 [[ 0.08838835  0.08838835 -0.08838835 -0.08838835]
 [ 0.08838835  0.08838835 -0.08838835 -0.08838835]
 [-0.08838835 -0.08838835  0.08838835  0.08838835]
 [-0.08838835 -0.08838835  0.08838835  0.08838835]] 

Member 2: 
 [[ 0.   -0.   -0.    0.  ]
 [-0.    0.25  0.   -0.25]
 [-0.    0.    0.   -0.  ]
 [ 0.   -0.25 -0.    0.25]] 

Member 3: 
 [[ 0.072 -0.096 -0.072  0.096]
 [-0.096  0.128  0.096 -0.128]
 [-0.072  0.096  0.072 -0.096]
 [ 0.096 -0.128 -0.096  0.128]]


In [21]:
# The size of a resultant matrix depends on the amount of members being factored, 
# so the function needs to have an if statement that takes this into consideration.

def stiff_matrix(xy1, xy2, other_dofs=None):
    """ Given the coordinates of a member's nodes, its stiffness matrix is calculated. 
    There is also the option to calculate a tuple which includes other degrees 
    of freedom in order to produce a matrix for the entire system.
    """
    mem_len = member_len(xy1, xy2)
    lmx, lmy = dir_cos(xy1, xy2)
    k_arr = np.array(
        [
        [lmx**2, lmx*lmy, -lmx**2, -lmx*lmy],
        [lmx*lmy, lmy**2, -lmx*lmy, -lmy**2],
        [-lmx**2, -lmx*lmy, lmx**2, lmx*lmy],
        [-lmx*lmy, -lmy**2, lmx*lmy, lmy**2]
        ]
    ) / mem_len
    """Checks for degrees of freedom and inserts rows/cols of zeroes if condition is met
    """
    if other_dofs:
        for i in other_dofs:
            k_arr = np.insert(k_arr, i-1, 0, axis=0)
            k_arr = np.insert(k_arr, i-1, 0, axis=1)
    return k_arr


k1 = stiff_matrix((x1, y1), (x2, y2), (5, 6, 7, 8, ))
k2 = stiff_matrix((x1, y1), (x3, y3), (3, 4, 7, 8, ))
k3 = stiff_matrix((x1, y1), (x4, y4), (3, 4, 5, 6, ))

print(k1, '\n')
print(k2, '\n')
print(k3)

[[ 0.08838835  0.08838835 -0.08838835 -0.08838835  0.          0.
   0.          0.        ]
 [ 0.08838835  0.08838835 -0.08838835 -0.08838835  0.          0.
   0.          0.        ]
 [-0.08838835 -0.08838835  0.08838835  0.08838835  0.          0.
   0.          0.        ]
 [-0.08838835 -0.08838835  0.08838835  0.08838835  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]] 

[[ 0.   -0.    0.    0.   -0.    0.    0.    0.  ]
 [-0.    0.25  0.    0.    0.   -0.25  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [-0.    0.    0.    0.    0.   -0.    0.    0.  

$$
\mathbf{Q} = \mathbf{K} \, \mathbf{D}
$$

$$
\begin{bmatrix}
Q_1 \\
Q_2 \\
... \\
Q_n
\end{bmatrix}
= 
\begin{bmatrix}
\sum(\lambda x)^2_1 & \sum(\lambda x \lambda y)_2 & ... \\
\sum(\lambda x \lambda y)_1 & \sum(\lambda y)^2_2 & ... \\
... & ... & ... \\
... & ... & ...
\end{bmatrix}
\begin{bmatrix}
D_1 \\
D_2 \\
... \\
D_n
\end{bmatrix}
$$

In [22]:
# Add "sub-matrices" together to get resultant matrix
k_res = k1 + k2 + k3
k_res

array([[ 0.16038835, -0.00761165, -0.08838835, -0.08838835,  0.        ,
         0.        , -0.072     ,  0.096     ],
       [-0.00761165,  0.46638835, -0.08838835, -0.08838835,  0.        ,
        -0.25      ,  0.096     , -0.128     ],
       [-0.08838835, -0.08838835,  0.08838835,  0.08838835,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-0.08838835, -0.08838835,  0.08838835,  0.08838835,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -0.25      ,  0.        ,  0.        ,  0.        ,
         0.25      ,  0.        ,  0.        ],
       [-0.072     ,  0.096     ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.072     , -0.096     ],
       [ 0.096     , -0.128     ,  0.        ,  0.        ,  0.        ,
         0.        , -0.096     ,  0.128     ]])

There is a 500 lb horizontal load on node 1 to the left.

$$
\begin{bmatrix}
-500 \\
0 \\
Q_3 \\
Q_4 \\
Q_5 \\
Q_6 \\
Q_7 \\
Q_8 \\
\end{bmatrix}
= 
\begin{bmatrix}
\mathbf{0.160} & \mathbf{-0.008} & -0.088 & -0.088 & 0 & 0 & -0.072 & 0.096 \\
\mathbf{-0.008} & \mathbf{0.466} & -0.088 & -0.088 & 0 & -0.25 & 0.096 & -0.128 \\
\mathbf{-0.088} & \mathbf{-0.088} & 0.088 & 0.088 & 0 & 0 & 0 & 0 \\
\mathbf{-0.088} & \mathbf{-0.088} & 0.088 & 0.088 & 0 & 0 & 0 & 0 \\
\mathbf{0} & \mathbf{0} & 0 & 0 & 0 & 0 & 0 & 0 \\
\mathbf{0} & \mathbf{-0.25} & 0 & 0 & 0 & 0.25 & 0 & 0 \\
\mathbf{-0.072} & \mathbf{0.096} & 0 & 0 & 0 & 0 & 0.072 & -0.096 \\
\mathbf{0.096} & \mathbf{-0.128} & 0 & 0 & 0 & 0 & -0.096 & 0.128 \\
\end{bmatrix}
\begin{bmatrix}
D_1 \\
D_2 \\
D_3 \\
0 \\
0 \\
0 \\
0 \\
0 \\
\end{bmatrix}
$$

The focus is on the first two degrees of freedom.

$$
M
=
\begin{bmatrix}
\mathbf{0.160} & \mathbf{-0.008} \\
\mathbf{-0.008} & \mathbf{0.466} \\
\end{bmatrix}
$$

This section *M* of the matrix *K* can be used in the rearranged equation shown below to solve for D.
$$
\mathbf{Q} = \mathbf{K} \mathbf{D} → \mathbf{Q} = \mathbf{M} \mathbf{D} → \mathbf{D} = \mathbf{M^{-1}} \mathbf{Q}
$$

In [29]:
# Section M of the matrix K
# first two degrees of freedom

m_mtx = k_res[:2, :2]
m_mtx

array([[ 0.16038835, -0.00761165],
       [-0.00761165,  0.46638835]])

In [31]:
# Q matrix

q_mtx = [
         [-500],
         [0],
]

q_mtx

[[-500], [0]]

In [32]:
# Solving for the value of D

d_val = np.linalg.inv(m_mtx).dot(q_mtx)
d_val

array([[-3119.84986285],
       [  -50.91725097]])

Using the lower partition of the stiffness matrix, the reactions at the supports can be calculated.

In [33]:
# Last 6 rows of first two columns of matrix K

k_lower = k_res[2:, :2]
k_lower

array([[-0.08838835, -0.08838835],
       [-0.08838835, -0.08838835],
       [ 0.        ,  0.        ],
       [ 0.        , -0.25      ],
       [-0.072     ,  0.096     ],
       [ 0.096     , -0.128     ]])

In [34]:
q_val = k_lower.dot(d_val)
q_val

array([[ 280.25886597],
       [ 280.25886597],
       [   0.        ],
       [  12.72931274],
       [ 219.74113403],
       [-292.98817871]])

After finding the values for *Q*, the forces in each of the members can be determined.

$$
q_j = \dfrac{AE}{L}
\begin{bmatrix}
-\lambda x & -\lambda y & \lambda x & \lambda y
\end{bmatrix}
\begin{bmatrix}
DNx \\
DNy \\
DFx \\
DFy
\end{bmatrix}
$$

In [36]:
# Force in member 1
q1 = (np.array([-mem_x1, -mem_y1, mem_x1, mem_y1])/mem1).dot([d_val[0], d_val[1], [0], [0]])

# Force in member 2
q2 = (np.array([-mem_x2, -mem_y2, mem_x2, mem_y2])/mem2).dot([d_val[0], d_val[1], [0], [0]])

# Force in member 3
q3 = (np.array([-mem_x3, -mem_y3, mem_x3, mem_y3])/mem3).dot([d_val[0], d_val[1], [0], [0]])

print('Force in Member 1:', q1, '\n')
print('Force in Member 2:', q2, '\n')
print('Force in Member 3:', q3)

Force in Member 1: [-396.34588923] 

Force in Member 2: [-12.72931274] 

Force in Member 3: [366.23522339]
